# Import

In [ ]:
%pip install pytorch-tabnet

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

import torch
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# Data Load

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np

from google.colab import files
uploaded = files.upload()

import io
for filename in uploaded.keys():
  train = pd.read_csv(io.BytesIO(uploaded[filename]))
  print(f"Loaded file: {filename}")
  break

train.head()

from google.colab import files
uploaded = files.upload()

import io
for filename in uploaded.keys():
  test = pd.read_csv(io.BytesIO(uploaded[filename]))
  print(f"Loaded file: {filename}")
  break

test.head()

from sklearn.preprocessing import LabelEncoder

# 설립연도 타입 변환, 범주형으로 사용하기 위해 (int -> object)
train['설립연도'] =train['설립연도'].astype('object')
test['설립연도'] =test['설립연도'].astype('object')

category_features = ['설립연도','국가','분야','투자단계']
numeric_features = ['직원 수','고객수(백만명)','총 투자금(억원)','연매출(억원)','SNS 팔로워 수(백만명)']
bool_features = ['인수여부','상장여부']

# LabelEncoder 객체를 각 범주형 feature별로 따로 저장하여 사용
encoders = {}

# 범주형 데이터를 encoding, 범주형 데이터를 숫자로 변환
for feature in category_features:
    encoders[feature] = LabelEncoder()
    train[feature] = train[feature].fillna('Missing')
    test[feature] = test[feature].fillna('Missing')
    train[feature] = encoders[feature].fit_transform(train[feature])
    test[feature] = encoders[feature].transform(test[feature])

# 불리언 값을 0과 1로 변환 ('Yes' → 1, 'No' → 0 으로 변환)
bool_map = {'Yes': 1, 'No': 0}

for feature in bool_features:
    train[feature] = train[feature].map(bool_map)
    test[feature] = test[feature].map(bool_map)

# 수치형 변수 결측치를 평균값으로 대체
for feature in numeric_features:
    mean_value = train[feature].mean()
    train[feature] = train[feature].fillna(mean_value).infer_objects(copy=False)
    test[feature] = test[feature].fillna(mean_value).infer_objects(copy=False)

def parse_value_range(value):
    if isinstance(value, str) and '-' in value:
        low, high = map(float, value.split('-'))
        return (low + high) / 2
    try:
        return float(value)
    except:
        return np.nan

# '~이상' 한글 포함 문자열 처리
def parse_value_range(value):
    try:
        return float(value)
    except:
        if isinstance(value, str) and '이상' in value:
          return float(value.replace('이상', '').strip())
        return np.nan

# 기업가치를 중간값으로 변환
train['기업가치(중간값)'] = train['기업가치(백억원)'].apply(parse_value_range)
test['기업가치(중간값)'] = test['기업가치(백억원)'].apply(parse_value_range)

# 수치형 변수 ; 파생 변수 생성
def create_derived_features(df):
  df = df.copy()
  df['직원 1인당 매출'] = df['연매출(억원)'] / df['직원 수']
  df['고객 1인당 매출'] = df['연매출(억원)'] / df['고객수(백만명)']
  df['고객 1인당 투자금'] = df['총 투자금(억원)'] / df['고객수(백만명)']
  df['투자 대비 가치'] = df['기업가치(중간값)'] / df['총 투자금(억원)']
  df['직원 1인당 기업가치'] = df['기업가치(중간값)'] / df['직원 수']
  df['투자금 대비 매출'] = df['연매출(억원)'] / df['총 투자금(억원)']
  df['SNS비율'] = df['SNS 팔로워 수(백만명)'] / df['고객수(백만명)']
  df['직원 1인당 투자금'] = df['총 투자금(억원)'] / df['직원 수']
  df['고객 1인당 기업가치'] = df['기업가치(중간값)'] / df['고객수(백만명)']
  df['고객 1인당 SNS 접점 수'] = df['SNS 팔로워 수(백만명)'] / df['고객수(백만명)']
  df['고객 대비 직원 수 비율'] = df['직원 수'] / df['고객수(백만명)']
  df['투자금 대비 SNS 팔로워'] = df['SNS 팔로워 수(백만명)'] / df['총 투자금(억원)']
  df['연매출 대비 기업가치'] = df['기업가치(중간값)'] / df['연매출(억원)']
  df['투자금 대비 기업가치'] = df['기업가치(중간값)'] / df['총 투자금(억원)']

  df.fillna(0, inplace=True) # NaN 값은 0 또는 다른 기준으로 대체
  return df

# train/test 데이터에 위 파생 변수들을 실제로 적용
train = create_derived_features(train)
test = create_derived_features(test)


# 상관관계 분석 코드
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

np.random.seed(2023)

# 새로운 변수로, 계산할 변수 선택
selected_vars = [
    '직원 수', '고객수(백만명)', '총 투자금(억원)', '연매출(억원)', 'SNS 팔로워 수(백만명)',
    '직원 1인당 매출', '고객 1인당 매출', '고객 1인당 투자금', '투자 대비 가치',
    '직원 1인당 기업가치', '투자금 대비 매출', 'SNS비율', '직원 1인당 투자금',
    '고객 1인당 기업가치', '고객 1인당 SNS 접점 수', '고객 대비 직원 수 비율',
    '투자금 대비 SNS 팔로워', '연매출 대비 기업가치', '투자금 대비 기업가치', '성공확률'
]

df_selected = train[selected_vars]
correlation_matrix = df_selected.corr()

# 상관관계를 절댓값 기준 내림차순 정렬
target_corr = correlation_matrix['성공확률'].drop('성공확률')
target_corr_sorted = target_corr.reindex(target_corr.abs().sort_values(ascending=False).index)

# (히트맵으로 시각화?)
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix with 성공확률 포함')
plt.tight_layout()
plt.savefig("Selected_Correlation_Heatmap.png")
plt.close()


# 상관관계 출력 코드
df = train.copy()

numeric_df = df.select_dtypes(include=[np.number])

correlation_with_success = numeric_df.corr()['성공확률'].drop('성공확률')

# 수치형 변수와 성공확률 간 상관관계 계산
correlation_with_success_sorted = correlation_with_success.reindex(
    correlation_with_success.abs().sort_values(ascending=False).index
)

print("성공확률과의 상관관계 (절댓값 내림차순):")
print(correlation_with_success_sorted)

#특성과 타겟 변수 분리
train = train.drop(columns=['ID'], axis = 1)
test = test.drop(columns=['ID'], axis = 1)

Saving train.csv to train (6).csv
Loaded file: train (6).csv


Saving test.csv to test (4).csv
Loaded file: test (4).csv


<ipython-input-9-45bf21bf1008>:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train[feature] = train[feature].fillna('Missing')
<ipython-input-9-45bf21bf1008>:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test[feature] = test[feature].fillna('Missing')
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 51649 (\N{HANGUL SYLLABLE JIG}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 50896 (\N{HANGUL SYLLABLE WEON}) missing from font(s) 

성공확률과의 상관관계 (절댓값 내림차순):
기업가치(중간값)          0.043256
상장여부               0.031413
국가                 0.029570
분야                -0.026773
직원 1인당 기업가치        0.022197
연매출(억원)           -0.018994
SNS 팔로워 수(백만명)     0.017927
SNS비율              0.017731
고객 1인당 SNS 접점 수    0.017731
투자금 대비 기업가치       -0.016067
투자 대비 가치          -0.016067
총 투자금(억원)         -0.014939
투자금 대비 매출         -0.014520
고객 대비 직원 수 비율      0.014196
고객수(백만명)          -0.012123
인수여부              -0.012065
직원 수              -0.011581
고객 1인당 기업가치        0.011152
고객 1인당 매출          0.009432
직원 1인당 매출          0.008515
직원 1인당 투자금        -0.008057
투자단계              -0.007835
고객 1인당 투자금         0.007661
연매출 대비 기업가치       -0.006579
설립연도               0.002147
투자금 대비 SNS 팔로워     0.001101
Name: 성공확률, dtype: float64


# K-Fold Model Training

In [ ]:
# 타겟 지정
target = train['성공확률']
X = train[features]
y = target

# KFold 설정
N_FOLDS = 5
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

models = [] # 모델 저장 리스트
cv_scores = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
    print(f"\n🔁 Fold {fold+1}/{N_FOLDS}")

    X_train = X.iloc[train_idx].values
    y_train = y.iloc[train_idx].values.reshape(-1, 1)

    X_valid = X.iloc[valid_idx].values
    y_valid = y.iloc[valid_idx].values.reshape(-1, 1)

    # 비지도 사전학습
    print("▶ Pretraining...")

    pretrainer = TabNetPretrainer(
        cat_idxs=cat_idxs,
        cat_dims=cat_dims,
        seed=42,
        verbose=0
    )

    pretrainer.fit(
        X_train=X_train,
        max_epochs=100,
        batch_size=512,
        virtual_batch_size=64
    )

    # 지도 학습
    print("▶ Fine-tuning...")
    model = TabNetRegressor(
        cat_idxs=cat_idxs,
        cat_dims=cat_dims,
        seed=42,
        verbose=0,
        optimizer_fn=torch.optim.AdamW
    )

    model.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_valid, y_valid)],
        from_unsupervised=pretrainer,
        eval_metric=['mae'],
        max_epochs=100,
        patience=10
    )

    # 모델을 메모리에 저장
    models.append(model)
    cv_scores.append(model.best_cost)

print("\n✅ 모든 fold 모델 학습 완료!")


🔁 Fold 1/5
▶ Pretraining...


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


▶ Fine-tuning...


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")



Early stopping occurred at epoch 33 with best_epoch = 23 and best_val_0_mae = 0.20616

🔁 Fold 2/5
▶ Pretraining...


...

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


▶ Fine-tuning...


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")



Early stopping occurred at epoch 51 with best_epoch = 41 and best_val_0_mae = 0.20543

✅ 모든 fold 모델 학습 완료!


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


# K-Fold Model Prediction

In [ ]:
# 저장된 모델들로 예측
predictions_list = []

for fold, model in enumerate(models):
    print(f"Predict with fold {fold+1}")
    preds = model.predict(test[features].values)
    predictions_list.append(preds)

# 평균 예측
final_predictions = np.mean(predictions_list, axis=0)

Predict with fold 1
Predict with fold 2
Predict with fold 3
Predict with fold 4
Predict with fold 5


# Submission

In [ ]:
sample_submission['성공확률'] = final_predictions
sample_submission.to_csv('./baseline_submission.csv', index = False, encoding = 'utf-8-sig')